# Task 13.4: LangGraph Workflow - Solutions

This notebook contains complete solutions for the LangGraph exercises.

---

## Challenge: Complex Content Creation Workflow

**Solution:** A workflow with multiple content types, research phase, and manager approval.

In [ ]:
from typing import TypedDict, Literal, List
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.llms import Ollama
from datetime import datetime

# Initialize LLM
llm = Ollama(model="llama3.1:8b", temperature=0.7)

# Define complex state
class AdvancedContentState(TypedDict):
    # Input
    topic: str
    content_type: str  # blog, tweet, email, linkedin
    urgency: str  # low, medium, high
    
    # Research phase
    research_notes: str
    key_points: List[str]
    
    # Writing phase
    draft: str
    revised_draft: str
    
    # Review phase
    writer_feedback: str
    editor_feedback: str
    manager_feedback: str
    
    # Approvals
    editor_approved: bool
    manager_approved: bool
    
    # Metadata
    revision_count: int
    messages: List[str]
    final_content: str

print("AdvancedContentState defined with multiple approval stages!")

In [ ]:
# Define nodes

def research_topic(state: AdvancedContentState) -> AdvancedContentState:
    """Research the topic before writing."""
    print("📚 Researching topic...")
    
    prompt = f"""Research the topic: {state['topic']}
    
Provide:
1. Three key facts or statistics
2. Main points to cover
3. Target audience considerations for {state['content_type']}

Keep it brief but informative."""
    
    research = llm.invoke(prompt)
    
    state['research_notes'] = research
    state['messages'].append(f"Research completed at {datetime.now().strftime('%H:%M:%S')}")
    
    return state

def select_content_type(state: AdvancedContentState) -> AdvancedContentState:
    """Prepare content-type specific guidelines."""
    print(f"📝 Preparing for {state['content_type']} content...")
    
    guidelines = {
        'blog': 'Write 300-500 words with headers, engaging intro, actionable conclusion.',
        'tweet': 'Write under 280 characters, punchy, include relevant hashtags.',
        'linkedin': 'Write 150-300 words, professional tone, include call to action.',
        'email': 'Write clear subject line, body, and call to action.',
    }
    
    state['key_points'] = [guidelines.get(state['content_type'], guidelines['blog'])]
    return state

def write_draft(state: AdvancedContentState) -> AdvancedContentState:
    """Write the initial draft."""
    print("✍️ Writing draft...")
    
    prompt = f"""Write a {state['content_type']} about: {state['topic']}

Research notes:
{state['research_notes']}

Guidelines:
{state['key_points'][0]}

Write only the content, no preamble."""
    
    draft = llm.invoke(prompt)
    state['draft'] = draft
    state['messages'].append("Draft written")
    
    return state

def editor_review(state: AdvancedContentState) -> AdvancedContentState:
    """Editor reviews the content."""
    print("📋 Editor reviewing...")
    
    current = state.get('revised_draft') or state['draft']
    
    prompt = f"""Review this {state['content_type']} as an editor:

{current}

If it's publication-ready, respond with: APPROVED: [brief praise]
If it needs work, respond with: REVISE: [specific feedback]"""
    
    feedback = llm.invoke(prompt)
    state['editor_feedback'] = feedback
    state['editor_approved'] = "APPROVED" in feedback.upper()
    
    return state

def manager_review(state: AdvancedContentState) -> AdvancedContentState:
    """Manager gives final approval."""
    print("👔 Manager reviewing...")
    
    current = state.get('revised_draft') or state['draft']
    
    # Simulate manager review based on urgency
    if state['urgency'] == 'high':
        # High urgency = auto-approve if editor approved
        state['manager_approved'] = True
        state['manager_feedback'] = "APPROVED: Expedited due to high urgency."
    else:
        # Normal review
        prompt = f"""As a manager, review this {state['content_type']}:

{current}

Does it align with brand guidelines and business goals?
Respond: APPROVED or REVISE with brief feedback."""
        
        feedback = llm.invoke(prompt)
        state['manager_feedback'] = feedback
        state['manager_approved'] = "APPROVED" in feedback.upper()
    
    return state

def revise_draft(state: AdvancedContentState) -> AdvancedContentState:
    """Revise based on feedback."""
    print(f"📝 Revising (attempt {state['revision_count'] + 1})...")
    
    feedback = state.get('manager_feedback') or state.get('editor_feedback', '')
    current = state.get('revised_draft') or state['draft']
    
    prompt = f"""Revise this {state['content_type']} based on feedback:

Current content:
{current}

Feedback:
{feedback}

Provide only the revised content."""
    
    state['revised_draft'] = llm.invoke(prompt)
    state['revision_count'] += 1
    
    return state

def publish(state: AdvancedContentState) -> AdvancedContentState:
    """Publish the final content."""
    print("🎉 Publishing!")
    
    state['final_content'] = state.get('revised_draft') or state['draft']
    state['messages'].append(f"Published at {datetime.now().strftime('%H:%M:%S')}")
    
    return state

print("All nodes defined!")

In [ ]:
# Define routing functions

def route_after_editor(state: AdvancedContentState) -> Literal["manager_review", "revise"]:
    if state['editor_approved']:
        return "manager_review"
    return "revise"

def route_after_manager(state: AdvancedContentState) -> Literal["publish", "revise"]:
    if state['manager_approved']:
        return "publish"
    if state['revision_count'] >= 3:
        # Force publish after max revisions
        return "publish"
    return "revise"

print("Routing functions defined!")

In [ ]:
# Build the graph
workflow = StateGraph(AdvancedContentState)

# Add nodes
workflow.add_node("research", research_topic)
workflow.add_node("prepare", select_content_type)
workflow.add_node("write", write_draft)
workflow.add_node("editor_review", editor_review)
workflow.add_node("manager_review", manager_review)
workflow.add_node("revise", revise_draft)
workflow.add_node("publish", publish)

# Add edges
workflow.set_entry_point("research")
workflow.add_edge("research", "prepare")
workflow.add_edge("prepare", "write")
workflow.add_edge("write", "editor_review")

workflow.add_conditional_edges(
    "editor_review",
    route_after_editor,
    {"manager_review": "manager_review", "revise": "revise"}
)

workflow.add_conditional_edges(
    "manager_review",
    route_after_manager,
    {"publish": "publish", "revise": "revise"}
)

workflow.add_edge("revise", "editor_review")  # Go back to editor after revision
workflow.add_edge("publish", END)

# Compile
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("Advanced workflow compiled!")
print("Flow: Research → Prepare → Write → Editor Review → Manager Review → Publish")

In [ ]:
# Test the advanced workflow
initial_state = {
    "topic": "Why AI Agents are the Future of Automation",
    "content_type": "linkedin",
    "urgency": "medium",
    "research_notes": "",
    "key_points": [],
    "draft": "",
    "revised_draft": "",
    "writer_feedback": "",
    "editor_feedback": "",
    "manager_feedback": "",
    "editor_approved": False,
    "manager_approved": False,
    "revision_count": 0,
    "messages": [],
    "final_content": ""
}

config = {"configurable": {"thread_id": "advanced-demo"}}

print("Running advanced workflow...")
print("="*60)

result = app.invoke(initial_state, config)

print("\n" + "="*60)
print("FINAL CONTENT")
print("="*60)
print(result['final_content'])

print(f"\nRevisions made: {result['revision_count']}")
print(f"Workflow log: {result['messages']}")

## Key Takeaways

1. **Multi-stage approval:** Research → Editor → Manager provides quality gates
2. **Content-type adaptation:** Different guidelines for different formats
3. **Urgency handling:** High-priority items can skip certain reviews
4. **Feedback loops:** Revisions cycle back through reviews
5. **Max revision limits:** Prevent infinite loops